![LangChain](img/langchain.jpeg)

Les systèmes **RAG (Retrieval-Augmented Generation)** dans LangChain permettent aux modèles de langage de s’appuyer sur des **connaissances externes** pour produire des réponses plus précises, actualisées et pertinentes.

Contrairement à un simple LLM qui génère une réponse uniquement à partir de ce qu’il a appris pendant son entraînement, un système RAG interroge une base de documents pour retrouver des morceaux d’information pertinents – appelés **chunks** – et les injecte dans le prompt du LLM.

![RAG](img/rag.jpeg)

**Que montre le schéma ci-dessus ?**

Le processus se divise en **deux grandes phases** : **préparation des documents** et **traitement des requêtes**.

**Préparation des documents (à gauche)**
- (1) Un fichier (document source) est divisé en **chunks**, c’est-à-dire en petits segments de texte.
- (2) Chaque chunk est passé dans un LLM Embedder, un encodeur qui transforme le texte en un vecteur numérique (**embeddings**).
- (3) Ces vecteurs sont ensuite stockés dans un Vector Store, une base de données spécialisée pour les recherches par **similarité sémantique**.

**Traitement des requêtes (à droite)**
- (a) Lorsqu’un utilisateur emet une requête, celle-ci est à son tour encodée via **le même LLM Embedder** pour obtenir son vecteur.
- (b) Ce vecteur est utilisé par le **Retriever**, qui compare la requête aux vecteurs des **chunks** pour trouver les plus similaires.
- (c) Les chunks retrouvés sont envoyés au LLM, qui les utilise comme contexte pour formuler une réponse.


En résumé, ce fonctionnement est illustré par la boucle :

> Requête → Encodage → Recherche dans la base vectorielle → Récupération des chunks → Passage au LLM → Réponse contextuelle


# 1. Chargement du modèle
___

## LLM local :

Dans cette section, nous chargeons un modèle de langage local grâce à **Ollama**. Cela permet de travailler avec un **LLM directement sur notre machine**, sans connexion à une API externe.

Nous utilisons ici la classe `ChatOllama` de **LangChain**, qui nous permet d’interagir facilement avec un modèle comme llama3 déjà téléchargé via Ollama.

GTP-OSS:20b, Mistral-Small3.2 24B GLM 4.7 Flash

## LLM Cloud :
Mistral

In [ ]:
import os
from IPython.display import display, Markdown, clear_output
from dotenv import load_dotenv
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma  # ✅ LangChain v1 : pip install langchain-chroma

from langchain_mistralai.chat_models import ChatMistralAI

# Chargement des clés d'API se trouvant dans le fichier .env.
# Ceci permet d'utiliser des modèles en ligne comme gpt-x, deepseek-x, etc...
load_dotenv(override=True)

model = ChatOllama(model="glm-4.7-flash")

#model = ChatMistralAI(model="mistral-large-latest", api_key=os.getenv("MISTRAL_API_KEY"))

# Modèle spécialisé pour convertir du texte en vecteurs (https://ollama.com/library/nomic-embed-text).
# Il existe d'autres modèles d'embeddings (comme "all-MiniLM-L6-v2", "text-embedding-ada-002", etc.)
# avec des performances et dimensions variées selon les cas d'usage (recherche sémantique, classification, etc.).
embedder = OllamaEmbeddings(model="nomic-embed-text")

# 2. RAG standard
___

Le **RAG standard** consiste à :
- formuler une requête explicite
- interroger une base de documents vectorisée
- utiliser un modèle LLM pour générer une réponse à partir des résultats retrouvés.

Ce pipeline est **efficace pour des questions indépendantes, sans contexte conversationnel**.

### 2.1 Préparation des documents

Nous initialisons les chemins nécessaires à la préparation des documents d’entrée.

In [3]:
# Récupère le chemin absolu du répertoire courant (là où le script est exécuté)
current_dir = os.getcwd()

# Nom du fichier texte contenant les comptes rendus de réunion
file_name = "meeting_reports.txt"

# Construit le chemin complet vers le fichier texte dans le dossier "data"
file_path = os.path.join(current_dir, "data", file_name)

# Définit le chemin du répertoire où sera stockée la base de données vectorielle (Chroma DB)
db_dir = os.path.join(current_dir, "data", "db")

### 2.2 Initialisation du vector store

Nous vérifions ici si la base vectorielle existe déjà.
Si ce n’est pas le cas, le fichier source est chargé, découpé en morceaux, enrichi de métadonnées, puis indexé dans Chroma DB.

In [4]:
if not os.path.exists(db_dir):
    print("Initializing vector store...")

    # Chargement du fichier texte brut contenant les documents
    loader = TextLoader(file_path)
    loaded_document = loader.load()

    # Découpage du document en chunks de 1000 caractères avec un chevauchement de 0
    # - chunk_size détermine la taille maximale de chaque morceau (en nombre de caractères ici : 1000)
    # - chunk_overlap permet de conserver un chevauchement entre les morceaux pour éviter les coupures abruptes, ici il est à 0, donc sans recouvrement.
    # - RecursiveCharacterTextSplitter est souvent préféré en pratique pour des documents textuels comme des comptes rendus,
    #   des articles ou de la documentation technique, car il garde mieux le contexte sémantique.
    #   Ce splitter tente d'abord de découper sur les sauts de ligne, puis sur les phrases, puis sur les mots, etc.
    # ... d'autres Text Splitter comme CharachterTextSplitter existent. À approfondir si besoin
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    chunks = text_splitter.split_documents(loaded_document)

    # Ajout de métadonnées à chaque chunk (utile pour le filtrage ou le suivi de provenance).
    # Ici 2 metadata sont ajoutés mais il pourrait en y avoir plus.
    for chunk in chunks:
        chunk.metadata["source"] = file_path    # Chemin d'origine du document
        chunk.metadata["category"] = "meeting"  # Catégorie de contenu (à adapter selon les besoins)

    # Création et persistance de la base vectorielle dans le dossier défini
    db = Chroma.from_documents(chunks, embedder, persist_directory=db_dir)

    print("Vector store created !")

### 2.3 Initialisation du moteur de recherche vectorielle

Une fois la base vectorielle Chroma initialisée avec les embeddings, nous la transformons en **moteur de recherche (retriever)**.
Cela permet de retrouver les documents les plus proches sémantiquement d’une question ou d’une requête.

In [5]:
# Chargement de la base vectorielle existante, avec liaison avec le même embedder ayant servi pour créer la base vectorielle
db = Chroma(persist_directory=db_dir, embedding_function=embedder)

# Conversion de la base Chroma en "retriever" pour effectuer des recherches par similarité
# - search_type="similarity" utilise la distance cosinus entre les vecteurs
# - "k": 3 signifie que l'on souhaite récupérer les 3 documents les plus proches
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# 💡 Il est aussi possible d’utiliser d’autres types de recherche (search_type) :
# - "mmr" (Maximal Marginal Relevance) : équilibre entre pertinence et diversité des résultats
# - "similarity_score_threshold" : retourne uniquement les documents dont le score dépasse un certain seuil
#      search_kwargs={"score_threshold": 0.8} permet par exemple de filtrer les résultats peu pertinents
#
# D’autres paramètres utiles dans search_kwargs :
# - "fetch_k" : nombre de documents à récupérer avant le tri final (utile avec MMR)
# - "lambda_mult" : pondération entre pertinence et diversité dans MMR
#
# Etc... à approfondir si besoin

/var/folders/59/q0j81gmx5h7gvpg38k8hknzw0000gn/T/ipykernel_1472/573588974.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=db_dir, embedding_function=embedder)


### 2.4 Exécution d’une requête de recherche

Dans cette étape, nous combinons la recherche vectorielle avec un LLM via une chaîne LCEL (LangChain Expression Language).

Le pipeline est composé de 4 étapes enchaînées avec l'opérateur `|` :
1. **Récupération** : le retriever cherche les chunks pertinents et une fonction les formate en texte
2. **Prompt** : le contexte récupéré et la question sont injectés dans un template structuré
3. **LLM** : le modèle génère une réponse à partir du prompt enrichi
4. **Parser** : la sortie brute du modèle est convertie en chaîne de caractères simple

Cette approche remplace le passage manuel des messages et garantit que le modèle répond **uniquement à partir des documents fournis**.

In [ ]:
# Requête posée par l'utilisateur
query = "Quels sont les réunions concernant la société Neolink ?"

# Optionnel : affichage manuel des chunks retrouvés (utile pour debug ou vérification)
# relevant_chunks = retriever.invoke(query)
# for i, chunk in enumerate(relevant_chunks, 1):
#     print(f"Chunk {i}:\n{chunk.page_content}\n")

# Fonction utilitaire pour formater les documents récupérés en un seul bloc de texte
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Template du prompt : le contexte récupéré et la question sont injectés dynamiquement
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Tu es un assistant qui aide à retrouver tout type d'informations interne à une entreprise. "
     "Réponds uniquement en te basant sur les documents fournis. "
     "Si l'information n'est pas dans les documents, dis-le clairement."),
    ("human", "Documents pertinents :\n\n{context}\n\nQuestion : {question}")
])

# Construction de la chaîne RAG avec LCEL :
# 1. retriever | format_docs : récupère les chunks et les formate en texte
# 2. RunnablePassthrough() : laisse passer la question telle quelle vers le prompt
# 3. prompt : injecte contexte + question dans le template
# 4. model : génère la réponse
# 5. StrOutputParser() : extrait le texte de la réponse (plus besoin de .content)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Exécution de la chaîne
result = rag_chain.invoke(query)

display(Markdown(result))

### 🧩 Exercice

La société NovTech gère de nombreux documents internes :
- des rapports d’incidents (panne, erreur technique, post-mortem),
- des procédures opérationnelles (onboarding, accès système, déploiement…).

Actuellement, les équipes perdent du temps à chercher les bonnes informations à travers des fichiers éparpillés.

Votre objectif est de construire un assistant basé sur l'architecture RAG qui permettra :
- de retrouver rapidement les procédures en cas de besoin,
- de consulter les résolutions d’incidents similaires,
- de répondre à des questions en langage naturel en s’appuyant uniquement sur les documents internes.

Pour vous aider, vous pouvez suivre les étapes suivantes :
1. Chargement des documents
2. Découpage en chunks
3. Indexation vectorielle
4. Recherche contextuelle
5. Génération de réponse

ℹ️ Les documents de l'entreprise se trouve dans le dossier `data/novtech`.
💪🏻 **Bonus** : Rendre possible un filtrage par catégorie dans les recherches

In [7]:
# Votre code ici

# 3. RAG conversationnel
___

Dans un cadre d'**interaction continue**, les utilisateurs posent souvent des questions implicites ou référentielles (ex. "Et lui ?"). Le **RAG conversationnel** ajoute une étape clé : la **reformulation de la question en prenant en compte l'historique du dialogue**.

Cette version de RAG permet de maintenir la pertinence des recherches dans la base vectorielle tout en conservant la fluidité de la conversation, ce qui la rend adaptée aux assistants IA ou aux chatbots avancés.

**Exemple**

Historique de la conversation :
- Utilisateur : *Qui est le CEO de Tesla ?*
- IA : *Elon Musk est le CEO de Tesla*.
- Utilisateur : *Et de SpaceX ?*

➡️ La question "Et de SpaceX ?" est ambiguë seule. Le moteur de recherche (retriever) ne sait pas de quoi il s'agit exactement.

Avec une reformulation de la question de l'utilisateur cela donnerait : "Qui est le CEO de SpaceX ?"

➡️ Résultat : la requête est claire, et la recherche dans la base vectorielle peut retourner les bons documents.

**👍 Approche LCEL avec historique explicite**

On étend la chaîne RAG standard en lui passant un `chat_history` (liste de `HumanMessage` / `AIMessage`) via un `MessagesPlaceholder`.
À chaque tour, le modèle dispose à la fois du contexte documentaire récupéré **et** de l'historique de la conversation, ce qui lui permet de reformuler et de répondre de façon cohérente.

La gestion de la mémoire reste sous le contrôle du développeur : simple liste Python mise à jour après chaque échange.

> 💡 Pour aller plus loin avec une mémoire persistante multi-sessions, voir la **section 4** de ce notebook.

In [ ]:
# Prompt avec historique de conversation intégré via MessagesPlaceholder
# Le modèle reçoit : le contexte documentaire + l'historique + la nouvelle question
prompt_conv = ChatPromptTemplate.from_messages([
    ("system",
     "Tu es un assistant qui aide à retrouver tout type d'informations interne à une entreprise. "
     "Réponds uniquement en te basant sur les documents fournis. "
     "Si l'information n'est pas dans les documents, dis-le clairement."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "Documents pertinents :\n\n{context}\n\nQuestion : {question}")
])

# Chaîne RAG conversationnelle avec LCEL
# Le retriever est appelé avec la question courante à chaque tour
conversational_rag_chain = (
    {
        "context": lambda x: "\n\n".join(doc.page_content for doc in retriever.invoke(x["question"])),
        "question": lambda x: x["question"],
        "chat_history": lambda x: x["chat_history"]
    }
    | prompt_conv
    | model
    | StrOutputParser()
)

# Historique de conversation (liste de HumanMessage / AIMessage)
chat_history = []

# Boucle de chat
# ⚠️ Changer `while False:` en `while True:` pour activer l'exemple
while False:
    user_input = input("Vous : ")
    clear_output(wait=True)
    display(Markdown(f"**Vous :** {user_input}"))

    if user_input.lower() in ["stop", "exit", "quit"]:
        print("Fin de la conversation.")
        break

    result = conversational_rag_chain.invoke({
        "question": user_input,
        "chat_history": chat_history
    })

    display(Markdown(result))

    # Mise à jour de l'historique pour le prochain tour
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=result))

### 🧩 Exercice

Repartez de l'exercice précédent (NovTech), et implémentez un assistant de conversation continue.

In [ ]:
# Votre code ici

# 4. RAG avec mémoire persistante (LangGraph)
___

**LangGraph** est le framework d'orchestration de LangChain pour construire des applications **stateful** (avec état persistant). Là où la section 3 gère l'historique de conversation manuellement via une liste Python, LangGraph prend en charge cette mémoire de façon **automatique, persistante et multi-sessions**.

Dans cette section, nous allons enrichir le RAG conversationnel avec :
- un **agent** capable de décider dynamiquement quand consulter la base documentaire,
- une **mémoire persistante** (`MemorySaver`) isolée par session via un `thread_id`,
- la possibilité de gérer **plusieurs conversations en parallèle** sans collision d'historique.

### Pourquoi LangGraph ?

Dans la section 3, l'historique est une liste Python gérée manuellement :
```python
chat_history.append(HumanMessage(...))
chat_history.append(AIMessage(...))
```
C'est suffisant pour une session simple, mais cette approche a des limites :
- L'historique **disparaît** à la fin du processus,
- Impossible de gérer **plusieurs sessions en parallèle** sans collision,
- Pas de possibilité de **reprendre** une conversation interrompue.

LangGraph résout ces problèmes avec `MemorySaver` et le concept de `thread_id`.

In [ ]:
from langchain.agents import create_agent
from langchain.tools import tool
from langgraph.checkpoint.memory import MemorySaver

### 4.1 Définition de l'outil de retrieval

L'outil est une fonction décorée avec `@tool` que l'agent peut appeler dynamiquement.
Le décorateur `response_format="content_and_artifact"` permet de retourner à la fois
le texte formaté (utilisé dans le prompt) et les documents bruts (pour le debug).

In [ ]:
@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Recherche des informations dans la base de documents internes de l'entreprise."""
    retrieved_docs = retriever.invoke(query)
    serialized = "\n\n".join(
        f"Source : {doc.metadata.get('source', 'inconnue')}\nContenu : {doc.page_content}"
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

### 4.2 Création de l'agent avec mémoire persistante

`MemorySaver` est un checkpoint en mémoire vive fourni par LangGraph.
Il associe automatiquement chaque échange à un `thread_id` : deux conversations
avec des `thread_id` différents ont des historiques totalement séparés.

In [ ]:
# Mémoire persistante : chaque session identifiée par son thread_id a son propre historique
memory = MemorySaver()

# L'agent décide dynamiquement à chaque tour s'il doit appeler l'outil de recherche
agent = create_agent(
    model,
    tools=[retrieve],
    system_prompt=(
        "Tu es un assistant qui aide à retrouver tout type d'informations interne à une entreprise. "
        "Utilise l'outil de recherche pour trouver des informations pertinentes avant de répondre. "
        "Si l'information n'est pas dans les documents, dis-le clairement."
    ),
    checkpointer=memory
)

### 4.3 Conversation avec gestion du thread_id

Le `thread_id` identifie la session. Pour démarrer une **nouvelle conversation**
sans historique, changez simplement la valeur de `thread_id`.

Deux threads peuvent tourner en parallèle sans interférence.

In [ ]:
# Identifiant de session — changer la valeur pour démarrer une nouvelle conversation
config = {"configurable": {"thread_id": "session_1"}}

# Boucle de chat
# ⚠️ Changer `while False:` en `while True:` pour activer l'exemple
while False:
    user_input = input("Vous : ")
    clear_output(wait=True)
    display(Markdown(f"**Vous :** {user_input}"))

    if user_input.lower() in ["stop", "exit", "quit"]:
        print("Fin de la conversation.")
        break

    # L'agent gère automatiquement : retrieval, historique, génération
    response = agent.invoke(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config
    )

    display(Markdown(response["messages"][-1].content))

### 4.4 Sessions multiples en parallèle

Avec un seul agent et `MemorySaver`, on peut simuler deux utilisateurs distincts
dont les historiques ne se mélangent pas.

In [ ]:
# Session utilisateur A
config_a = {"configurable": {"thread_id": "utilisateur_A"}}
# Session utilisateur B
config_b = {"configurable": {"thread_id": "utilisateur_B"}}

# Utilisateur A pose une première question
response_a1 = agent.invoke(
    {"messages": [{"role": "user", "content": "Quelles réunions concernent Neolink ?"}]},
    config=config_a
)
print("[Session A — Tour 1]")
display(Markdown(response_a1["messages"][-1].content))

# Utilisateur B pose une question indépendante
response_b1 = agent.invoke(
    {"messages": [{"role": "user", "content": "Quelles sont les décisions prises en janvier ?"}]},
    config=config_b
)
print("\n[Session B — Tour 1]")
display(Markdown(response_b1["messages"][-1].content))

# Utilisateur A pose une question de suivi — l'agent se souvient du contexte A
response_a2 = agent.invoke(
    {"messages": [{"role": "user", "content": "Et les participants à ces réunions ?"}]},
    config=config_a
)
print("\n[Session A — Tour 2 (question de suivi)]")
display(Markdown(response_a2["messages"][-1].content))

### 🧩 Exercice

Reprenez les documents NovTech (incidents et procédures) déjà utilisés dans les sections précédentes.

1. Créez un outil `@tool` de retrieval pointant sur la base NovTech.
2. Construisez un agent avec `create_agent` + `MemorySaver`.
3. Simulez **deux sessions utilisateur** (`thread_id` différents) posant des questions sur des incidents.
4. Vérifiez que les contextes ne se mélangent pas entre les sessions.

💪🏻 **Bonus** : Ajoutez un second outil `retrieve_by_category(query, category)` qui filtre les résultats par catégorie (`incidents` ou `procedures`) en utilisant les métadonnées Chroma.

In [ ]:
# Votre code ici